In [ ]:
import numpy as np
import pandas as pd
from scipy import signal
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import joblib
from collections import deque
import threading
import time
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configuration
CONFIG = {
    'sampling_rate': 360,  # Hz - matches MIT-BIH standard
    'window_size': 180,    # samples per beat (0.5 seconds)
    'risk_window_minutes': 10,  # time window for risk calculation
    'prediction_horizon_hours': 6,  # predict events within 6 hours
}

# Arrhythmia classes from MIT-BIH database
CLASSES = {0: 'Normal', 1: 'SVE', 2: 'VEB', 3: 'Fusion', 4: 'Unknown'}

# Clinical risk weights (higher = more dangerous)
RISK_WEIGHTS = {'Normal': 0.0, 'SVE': 2.0, 'VEB': 5.0, 'Fusion': 8.0, 'Unknown': 3.0}

In [ ]:
class ECGPreprocessor:
    def __init__(self, fs=360):
        self.fs = fs
        # Design filters for different noise types
        self.hp_filter = signal.butter(4, 0.5/(fs/2), btype='high', output='sos')  # Remove baseline drift
        self.lp_filter = signal.butter(4, 40/(fs/2), btype='low', output='sos')   # Remove muscle noise
        self.notch_filter = signal.iirnotch(50, 30, fs, output='sos')             # Remove power line interference

    def clean_signal(self, raw_ecg):
        """
        Clean raw ECG signal by removing noise

        Steps:
        1. Remove DC offset (center signal around zero)
        2. High-pass filter (remove breathing artifacts)
        3. Notch filter (remove 50Hz electrical interference)
        4. Low-pass filter (remove muscle artifacts)
        5. Wavelet denoising (advanced mathematical smoothing)
        """
        # Step 1: Remove DC offset
        signal_clean = raw_ecg - np.mean(raw_ecg)

        # Step 2: Remove baseline wander (breathing)
        signal_clean = signal.sosfilt(self.hp_filter, signal_clean)

        # Step 3: Remove power line interference
        signal_clean = signal.sosfilt(self.notch_filter, signal_clean)

        # Step 4: Remove high frequency noise
        signal_clean = signal.sosfilt(self.lp_filter, signal_clean)

        # Step 5: Wavelet denoising (removes remaining artifacts)
        signal_clean = self._wavelet_denoise(signal_clean)

        return signal_clean

    def _wavelet_denoise(self, signal_data, threshold=0.04):
        """
        Advanced denoising using Discrete Wavelet Transform
        Think of this as mathematical noise reduction
        """
        try:
            import pywt
            # Decompose signal into frequency components
            coeffs = pywt.wavedec(signal_data, 'sym8', level=6)

            # Apply soft thresholding to remove noise
            sigma = np.median(np.abs(coeffs[-1])) / 0.6745
            threshold_value = threshold * sigma

            coeffs_clean = []
            for i, coeff in enumerate(coeffs):
                if i == 0:  # Keep low frequency components
                    coeffs_clean.append(coeff)
                else:  # Clean high frequency components
                    coeffs_clean.append(pywt.threshold(coeff, threshold_value, mode='soft'))

            # Reconstruct clean signal
            return pywt.waverec(coeffs_clean, 'sym8')
        except:
            return signal_data  # Return original if wavelet not available

In [ ]:
class HeartbeatDetector:
    def __init__(self, fs=360, window_size=180):
        self.fs = fs
        self.window_size = window_size
        self.half_window = window_size // 2

    def find_heartbeats(self, clean_ecg):
        """
        Find individual heartbeats in ECG signal

        Process:
        1. Enhance QRS complexes (heartbeat spikes)
        2. Find peaks with adaptive thresholding
        3. Extract beat windows around each peak
        4. Quality control - reject noisy beats
        """
        # Step 1: Enhance QRS complexes using Pan-Tompkins algorithm
        r_peaks = self._detect_r_peaks(clean_ecg)

        # Step 2: Extract beat windows
        beats, peak_locations = self._extract_beat_windows(clean_ecg, r_peaks)

        return beats, peak_locations

    def _detect_r_peaks(self, signal_data):
        """
        Pan-Tompkins R-peak detection algorithm
        Enhanced version of classic heartbeat detection
        """
        # Differentiate signal (enhances sharp changes)
        diff_signal = np.diff(signal_data)

        # Square signal (emphasizes large changes)
        squared = diff_signal ** 2

        # Moving average (smooths the signal)
        window_samples = int(0.08 * self.fs)  # 80ms window
        smoothed = np.convolve(squared, np.ones(window_samples)/window_samples, mode='same')

        # Find peaks with adaptive thresholding
        peaks, _ = signal.find_peaks(
            smoothed,
            height=0.3 * np.max(smoothed),     # Must be 30% of max height
            distance=int(0.6 * self.fs),      # Minimum 600ms between beats (100 BPM max)
            prominence=0.1 * np.max(smoothed)  # Must stand out from surroundings
        )

        # Refine peak locations to original signal maxima
        refined_peaks = []
        for peak in peaks:
            # Search ±100ms around detected peak
            search_start = max(0, peak - int(0.1 * self.fs))
            search_end = min(len(signal_data), peak + int(0.1 * self.fs))

            # Find actual maximum in original signal
            local_segment = signal_data[search_start:search_end]
            local_max_idx = np.argmax(local_segment)
            actual_peak = search_start + local_max_idx

            refined_peaks.append(actual_peak)

        return np.array(refined_peaks)

    def _extract_beat_windows(self, signal_data, r_peaks):
        """
        Extract 180-sample windows around each R-peak
        """
        beats = []
        valid_peaks = []

        for peak in r_peaks:
            start_idx = peak - self.half_window
            end_idx = peak + self.half_window

            # Check if window is within signal bounds
            if start_idx >= 0 and end_idx < len(signal_data):
                beat_window = signal_data[start_idx:end_idx]

                # Quality control - only keep good beats
                if self._is_good_quality(beat_window):
                    beats.append(beat_window)
                    valid_peaks.append(peak)

        return np.array(beats), np.array(valid_peaks)

    def _is_good_quality(self, beat):
        """
        Quality control: reject noisy or invalid beats
        """
        # Check amplitude range (too flat = bad)
        if np.max(beat) - np.min(beat) < 0.1:
            return False

        # Check for excessive high-frequency noise
        noise_level = np.sum(np.abs(np.diff(beat, 2)))
        if noise_level > 10 * np.std(beat):
            return False

        # Check for reasonable beat morphology
        if np.any(np.isnan(beat)) or np.any(np.isinf(beat)):
            return False

        return True

In [ ]:
class BiLSTMFeatureExtractor:
    def __init__(self, input_shape=(180, 1)):
        self.input_shape = input_shape
        self.model = None
        self.scaler = StandardScaler()

    def build_network(self):
        """
        Build BiLSTM network for feature extraction

        Architecture:
        - Input: 180 ECG samples per beat
        - BiLSTM Layer 1: 128 units, bidirectional (looks forward and backward)
        - BiLSTM Layer 2: 128 units, bidirectional
        - Dense layers: Extract 7200 features
        - Output: Both features and classification
        """
        inputs = tf.keras.layers.Input(shape=self.input_shape)

        # Bidirectional LSTM layers (analyze beat in both directions)
        lstm1 = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(128, return_sequences=True, dropout=0.2)
        )(inputs)

        lstm2 = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(128, return_sequences=False, dropout=0.2)
        )(lstm1)

        # Feature extraction layers
        dense1 = tf.keras.layers.Dense(512, activation='relu')(lstm2)
        dropout1 = tf.keras.layers.Dropout(0.3)(dense1)

        # Main feature output (7200 features as per paper)
        features = tf.keras.layers.Dense(7200, activation='linear', name='features')(dropout1)

        # Classification output (for training)
        classifier_dense = tf.keras.layers.Dense(256, activation='relu')(dropout1)
        classifier = tf.keras.layers.Dense(5, activation='softmax', name='classifier')(classifier_dense)

        # Build model
        self.model = tf.keras.Model(inputs=inputs, outputs=[features, classifier])

        # Compile for training
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(0.001),
            loss={'features': 'mse', 'classifier': 'categorical_crossentropy'},
            loss_weights={'features': 0.3, 'classifier': 0.7},
            metrics={'classifier': 'accuracy'}
        )

        print("BiLSTM network built successfully")
        return self.model

    def extract_features(self, beats):
        """
        Extract 7200 features from each heartbeat

        Input: Array of beats (N x 180)
        Output: Array of features (N x 7200)
        """
        if self.model is None:
            raise ValueError("Model not built. Call build_network() first.")

        # Normalize beats (important for neural networks)
        beats_normalized = []
        for beat in beats:
            # Fit scaler to each beat individually for real-time processing
            beat_scaled = (beat - np.mean(beat)) / (np.std(beat) + 1e-8)
            beats_normalized.append(beat_scaled)

        # Reshape for LSTM input (N, 180, 1)
        beats_array = np.array(beats_normalized).reshape(-1, 180, 1)

        # Extract features (ignore classification output for feature extraction)
        features, _ = self.model.predict(beats_array, batch_size=32, verbose=0)

        return features

    def train_on_data(self, X_train, y_train, X_val, y_val, epochs=50):
        """
        Train the BiLSTM on labeled ECG data (like MIT-BIH)
        """
        if self.model is None:
            self.build_network()

        # Prepare training data
        X_train_scaled = np.array([
            (beat - np.mean(beat)) / (np.std(beat) + 1e-8)
            for beat in X_train
        ]).reshape(-1, 180, 1)

        X_val_scaled = np.array([
            (beat - np.mean(beat)) / (np.std(beat) + 1e-8)
            for beat in X_val
        ]).reshape(-1, 180, 1)

        # Create dummy feature targets (model will learn these)
        features_train = np.random.normal(0, 1, (len(X_train), 7200))
        features_val = np.random.normal(0, 1, (len(X_val), 7200))

        # Convert labels to categorical
        y_train_cat = tf.keras.utils.to_categorical(y_train, 5)
        y_val_cat = tf.keras.utils.to_categorical(y_val, 5)

        # Training callbacks
        callbacks = [
            tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
            tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
        ]

        # Train model
        history = self.model.fit(
            X_train_scaled,
            {'features': features_train, 'classifier': y_train_cat},
            validation_data=(X_val_scaled, {'features': features_val, 'classifier': y_val_cat}),
            epochs=epochs,
            batch_size=32,
            callbacks=callbacks,
            verbose=1
        )

        print(f"Training completed. Best validation accuracy: {max(history.history['val_classifier_accuracy']):.4f}")
        return history

In [ ]:
class FeatureCompressor:
    def __init__(self, n_components=150):
        self.n_components = n_components
        self.pca = PCA(n_components=n_components)
        self.fitted = False

    def fit_and_transform(self, training_features):
        """
        Learn compression from training data and apply it

        Input: Training features (N x 7200)
        Output: Compressed features (N x 150)
        """
        # Fit PCA on training data
        compressed_features = self.pca.fit_transform(training_features)
        self.fitted = True

        # Report how much information is retained
        variance_retained = np.sum(self.pca.explained_variance_ratio_)
        print(f"PCA compression: 7200 → 150 features")
        print(f"Information retained: {variance_retained:.1%}")

        return compressed_features

    def transform(self, features):
        """
        Apply learned compression to new data
        """
        if not self.fitted:
            raise ValueError("PCA not fitted. Call fit_and_transform() first.")

        return self.pca.transform(features)

    def save_model(self, filepath):
        """Save the trained PCA model"""
        joblib.dump(self.pca, filepath)
        print(f"PCA model saved to {filepath}")

    def load_model(self, filepath):
        """Load a trained PCA model"""
        self.pca = joblib.load(filepath)
        self.fitted = True
        print(f"PCA model loaded from {filepath}")

In [ ]:
class BeatClassifier:
    def __init__(self):
        # Random Forest: 100 decision trees voting
        self.classifier = RandomForestClassifier(
            n_estimators=100,      # 100 decision trees
            max_depth=10,          # Prevent overfitting
            min_samples_split=5,   # Need 5+ samples to split
            min_samples_leaf=2,    # Need 2+ samples in leaf
            random_state=42,       # Reproducible results
            n_jobs=-1             # Use all CPU cores
        )
        self.fitted = False

    def train(self, X_train, y_train):
        """
        Train Random Forest on compressed features

        Input: Compressed features (N x 150), Labels (N,)
        """
        print("Training Random Forest classifier...")
        self.classifier.fit(X_train, y_train)
        self.fitted = True

        # Report training accuracy
        train_accuracy = self.classifier.score(X_train, y_train)
        print(f"Training accuracy: {train_accuracy:.4f}")

    def predict_probabilities(self, features):
        """
        Get probability distribution over all classes

        Output: Probabilities for [Normal, SVE, VEB, Fusion, Unknown]
        """
        if not self.fitted:
            raise ValueError("Classifier not trained.")

        return self.classifier.predict_proba(features)

    def predict_classes(self, features):
        """Get most likely class for each beat"""
        if not self.fitted:
            raise ValueError("Classifier not trained.")

        return self.classifier.predict(features)

    def save_model(self, filepath):
        """Save trained classifier"""
        joblib.dump(self.classifier, filepath)
        print(f"Classifier saved to {filepath}")

    def load_model(self, filepath):
        """Load trained classifier"""
        self.classifier = joblib.load(filepath)
        self.fitted = True
        print(f"Classifier loaded from {filepath}")

In [ ]:
class CardiacRiskEngine:
    def __init__(self, window_minutes=10, prediction_hours=6):
        self.window_minutes = window_minutes
        self.prediction_hours = prediction_hours

        # Store recent beat predictions
        self.beat_history = deque(maxlen=window_minutes * 60)  # 1 beat per second max

        # Risk prediction model (trained on outcomes)
        self.risk_predictor = LogisticRegression()
        self.risk_model_trained = False

    def add_beat_prediction(self, timestamp, class_probabilities):
        """
        Add a new beat classification to history

        Input:
        - timestamp: when this beat occurred
        - class_probabilities: [P(Normal), P(SVE), P(VEB), P(Fusion), P(Unknown)]
        """
        # Calculate risk score for this individual beat
        beat_risk = sum(prob * RISK_WEIGHTS[CLASSES[i]]
                       for i, prob in enumerate(class_probabilities))

        # Store beat information
        beat_data = {
            'timestamp': timestamp,
            'probabilities': class_probabilities,
            'risk_score': beat_risk,
            'dominant_class': CLASSES[np.argmax(class_probabilities)]
        }

        self.beat_history.append(beat_data)

    def calculate_window_risk_features(self):
        """
        Calculate risk features from recent beat history

        This is WHERE THE MAGIC HAPPENS - turning beat classifications into risk features
        """
        if len(self.beat_history) < 10:
            return None  # Need minimum beats for analysis

        recent_beats = list(self.beat_history)

        # Count frequency of each arrhythmia type
        class_counts = {class_name: 0 for class_name in CLASSES.values()}
        risk_scores = []

        for beat in recent_beats:
            # Add to class counts based on most likely class
            class_counts[beat['dominant_class']] += 1
            risk_scores.append(beat['risk_score'])

        total_beats = len(recent_beats)

        # Calculate comprehensive risk features
        features = {
            # Frequency features (percentage of each arrhythmia type)
            'normal_frequency': class_counts['Normal'] / total_beats,
            'sve_frequency': class_counts['SVE'] / total_beats,
            'veb_frequency': class_counts['VEB'] / total_beats,
            'fusion_frequency': class_counts['Fusion'] / total_beats,
            'unknown_frequency': class_counts['Unknown'] / total_beats,

            # Risk burden features
            'mean_risk_score': np.mean(risk_scores),
            'max_risk_score': np.max(risk_scores),
            'std_risk_score': np.std(risk_scores),

            # Dangerous arrhythmia burden
            'high_risk_burden': sum(1 for score in risk_scores if score > 5.0) / total_beats,
            'critical_arrhythmia_rate': (class_counts['VEB'] + class_counts['Fusion']) / total_beats,

            # Trend analysis (is situation getting worse?)
            'risk_trend': self._calculate_trend(risk_scores),

            # Pattern complexity
            'arrhythmia_variety': sum(1 for count in class_counts.values() if count > 0) - 1,  # Subtract normal

            # Total analysis window
            'total_beats_analyzed': total_beats
        }

        return features

    def _calculate_trend(self, risk_scores):
        """
        Calculate if risk is increasing or decreasing over time
        Positive = getting worse, Negative = getting better
        """
        if len(risk_scores) < 5:
            return 0.0

        # Simple linear regression slope
        x = np.arange(len(risk_scores))
        slope, _ = np.polyfit(x, risk_scores, 1)
        return slope

    def predict_cardiac_event_risk(self):
        """
        Predict probability of cardiac event in next 6 hours

        Returns: Probability between 0.0 and 1.0
        """
        features = self.calculate_window_risk_features()
        if features is None:
            return 0.0

        if self.risk_model_trained:
            # Use trained ML model
            feature_vector = np.array([list(features.values())]).reshape(1, -1)
            event_probability = self.risk_predictor.predict_proba(feature_vector)[0][1]
            return event_probability
        else:
            # Use clinical rules if no trained model
            return self._rule_based_risk_assessment(features)

    def _rule_based_risk_assessment(self, features):
        """
        Clinical rule-based risk assessment
        Based on established cardiology guidelines
        """
        risk_score = 0.0

        # Rule 1: High VEB frequency (>10% is concerning)
        if features['veb_frequency'] > 0.10:
            risk_score += 0.4
        elif features['veb_frequency'] > 0.05:
            risk_score += 0.2

        # Rule 2: Any fusion beats (very concerning)
        if features['fusion_frequency'] > 0.02:  # >2%
            risk_score += 0.5

        # Rule 3: High overall arrhythmia burden
        if features['critical_arrhythmia_rate'] > 0.15:  # >15%
            risk_score += 0.3

        # Rule 4: Increasing risk trend (getting worse)
        if features['risk_trend'] > 0.1:
            risk_score += 0.3

        # Rule 5: Very high individual risk episodes
        if features['max_risk_score'] > 8.0:
            risk_score += 0.2

        # Rule 6: High risk burden over time
        if features['high_risk_burden'] > 0.20:  # >20% high-risk beats
            risk_score += 0.3

        # Rule 7: Multiple arrhythmia types (electrical instability)
        if features['arrhythmia_variety'] >= 3:
            risk_score += 0.2

        # Cap at 100%
        return min(risk_score, 1.0)

    def train_risk_model(self, historical_data):
        """
        Train ML risk model on historical patient data

        historical_data: List of cases with features and known outcomes
        Format: [{'features': {...}, 'had_event': True/False}, ...]
        """
        if len(historical_data) < 50:
            print("Need at least 50 cases for risk model training")
            return

        # Prepare training data
        X = []
        y = []

        for case in historical_data:
            X.append(list(case['features'].values()))
            y.append(1 if case['had_event'] else 0)

        X = np.array(X)
        y = np.array(y)

        # Train logistic regression
        self.risk_predictor.fit(X, y)
        self.risk_model_trained = True

        # Report performance
        accuracy = self.risk_predictor.score(X, y)
        print(f"Risk model trained on {len(historical_data)} cases")
        print(f"Training accuracy: {accuracy:.3f}")

In [ ]:
def train_complete_system():
    """
    Complete training pipeline for the cardiac prediction system
    """
    print("🎯 Starting complete system training...")

    # Step 1: Load MIT-BIH training data
    # (You'll need to implement this based on your data format)
    print("📊 Loading MIT-BIH Arrhythmia Database...")
    X_train, X_val, X_test, y_train, y_val, y_test = load_mitbih_data()

    # Step 2: Train BiLSTM feature extractor
    print("🧠 Training BiLSTM feature extractor...")
    feature_extractor = BiLSTMFeatureExtractor()
    feature_extractor.build_network()

    # Train on beat classification task
    history = feature_extractor.train_on_data(X_train, y_train, X_val, y_val, epochs=50)

    # Save BiLSTM weights
    feature_extractor.model.save_weights('models/bilstm_weights.h5')
    print("✅ BiLSTM training complete")

    # Step 3: Extract features for all data
    print("🔍 Extracting features from all data...")
    train_features = feature_extractor.extract_features(X_train)
    val_features = feature_extractor.extract_features(X_val)
    test_features = feature_extractor.extract_features(X_test)

    # Step 4: Train PCA compressor
    print("📏 Training PCA feature compressor...")
    compressor = FeatureCompressor(n_components=150)
    train_features_compressed = compressor.fit_and_transform(train_features)
    val_features_compressed = compressor.transform(val_features)
    test_features_compressed = compressor.transform(test_features)

    # Save PCA model
    compressor.save_model('models/pca_model.pkl')

    # Step 5: Train Random Forest classifier
    print("🌲 Training Random Forest classifier...")
    classifier = BeatClassifier()
    classifier.train(train_features_compressed, y_train)

    # Evaluate performance
    val_predictions = classifier.predict_classes(val_features_compressed)
    accuracy = np.mean(val_predictions == y_val)
    print(f"✅ Validation accuracy: {accuracy:.4f}")

    # Save classifier
    classifier.save_model('models/rf_classifier.pkl')

    # Step 6: Create synthetic risk training data
    # (In real deployment, you'd use historical patient outcomes)
    print("⚡ Training cardiac risk prediction model...")
    risk_engine = CardiacRiskEngine()

    # Generate synthetic outcomes based on arrhythmia patterns
    synthetic_risk_data = create_synthetic_risk_data(test_features_compressed, y_test)
    risk_engine.train_risk_model(synthetic_risk_data)

    print("🎉 Complete system training finished!")

    return {
        'bilstm_weights': 'models/bilstm_weights.h5',
        'pca_model': 'models/pca_model.pkl',
        'classifier': 'models/rf_classifier.pkl'
    }

def load_mitbih_data():
    """
    Load MIT-BIH Arrhythmia Database
    (Implement this based on your data source)
    """
    # This is a placeholder - implement based on your data format
    # You can download MIT-BIH from PhysioNet or use the Kaggle dataset
    # from the paper: https://www.kaggle.com/taejoongyoon/mitbit-arrhythmia-database

    path = kagglehub.dataset_download("taejoongyoon/mitbit-arrhythmia-database")
    return path

def create_synthetic_risk_data(features, labels):
    """
    Create synthetic risk training data
    (In production, use real patient outcomes)
    """
    risk_cases = []

    for i in range(len(features)):
        # Simulate risk features based on beat classifications
        window_features = {
            'normal_frequency': 0.8 if labels[i] == 0 else 0.4,
            'sve_frequency': 0.05 if labels[i] == 1 else 0.02,
            'veb_frequency': 0.15 if labels[i] == 2 else 0.03,
            'fusion_frequency': 0.10 if labels[i] == 3 else 0.01,
            'unknown_frequency': 0.05,
            'mean_risk_score': 4.0 if labels[i] > 0 else 1.0,
            'max_risk_score': 8.0 if labels[i] in [2, 3] else 3.0,
            'std_risk_score': 2.0,
            'high_risk_burden': 0.3 if labels[i] > 0 else 0.1,
            'critical_arrhythmia_rate': 0.25 if labels[i] in [2, 3] else 0.05,
            'risk_trend': 0.1 if labels[i] > 0 else -0.05,
            'arrhythmia_variety': 2 if labels[i] > 0 else 1,
            'total_beats_analyzed': 100
        }

        # Simulate cardiac event occurrence (higher for dangerous arrhythmias)
        event_probability = 0.15 if labels[i] in [2, 3] else 0.03
        had_event = np.random.random() < event_probability

        risk_cases.append({
            'features': window_features,
            'had_event': had_event
        })

    return risk_cases